In [1]:
from FME.interpolators.piecewiselinear_interpolator import PiecewiseLinearInterpolator as PLI
from FME.supports.tet_mesh import TetMesh
from FME.modelling.features.geological_feature import GeologicalFeatureInterpolator
# from FME.modelling.features.faulted_geological_feature import FaultedGeologicalFeature
from FME.visualisation.model_visualisation import LavaVuModelViewer
# from FME.modelling.structural_frame import StructuralFrameBuilder, StructuralFrame
# from FME.modelling.fault.fault_segment import FaultSegment
import numpy as np
import lavavu
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
dips = pd.read_csv('../Dips.csv',delimiter=';')
points_file_names = ['../APoints.csv','../BPoints.csv','../CPoints.csv','../DPoints.csv']
section_file_names = ['../ASection.csv','../BSection.csv','../CSection.csv','../DSection.csv']

# import all of the csv into the same dataframe
dfs = []
for f in points_file_names:
    dfs.append(pd.read_csv(f,delimiter=';'))
points = pd.concat(dfs,axis=0,ignore_index=True)
# sections
dfs = []
for f in section_file_names:
    dfs.append(pd.read_csv(f,delimiter=';'))
sections = pd.concat(dfs,axis=0,ignore_index=True)
# apoints = pd.read_csv('APoints.csv',delimiter=';')
# asection = pd.read_csv('ASection.csv',delimiter=';')



In [3]:
boundary_points = np.zeros((2,3))

boundary_points[0,0] = 548800
boundary_points[0,1] = 7816600
boundary_points[0,2] = -11010
boundary_points[1,0] = 552500
boundary_points[1,1] = 7822000
boundary_points[1,2] = -8400
mesh = TetMesh()
mesh.setup_mesh(boundary_points, nstep=1, n_tetra=150000,)


In [4]:
interpolator = PLI(mesh)
stratigraphy_builder = GeologicalFeatureInterpolator(
    interpolator=interpolator,
    name='stratigraphy')
solver = 'lu'
for i, r in points.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in sections.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in dips.iterrows():
    stratigraphy_builder.add_planar_constraint([r['X'],r['Y'],r['Z']],[r['OrientX'],r['OrientY'],r['OrientZ']])
# stratigraphy_builder.add_point([6.1,0.1,1.1],0.)
# stratigraphy_builder.add_point([6.1,0.1,2.1],1.)
# stratigraphy_builder.add_strike_and_dip([1,1,1],90.,0.)
unique = np.unique(stratigraphy_builder.interpolator.get_control_points()[:,3])

In [5]:
stratigraphy = stratigraphy_builder.build(solver=solver,
                                          cgw=30,
                                         cpw=35,
                                         gpw=35)#,cgw=100)


In [ ]:
viewer = LavaVuModelViewer(background="white")
viewer.plot_isosurface(stratigraphy, slices=unique,  colour='blue')
viewer.plot_value_data(stratigraphy.support.interpolator.get_control_points()[:,:3],
                       stratigraphy.support.interpolator.get_control_points()[:,3],
                       'val')
# viewer.plot_isosurface(stratigraphy, slices=[30,125,300], colour='red')
viewer.plot_vector_data(
    stratigraphy.support.interpolator.get_gradient_control()[:,:3],
    stratigraphy.support.interpolator.get_gradient_control()[:,3:],
    'grad')

viewer.interactive()

Creating isosurface for 0.000000
Creating isosurface for 60.000000
Creating isosurface for 250.000000


In [ ]:
viewer.lv.image("~/dev/python/FME/examples/Claudius/loop.png")